# Compare Multiple Regression Algorithms


## Step 1 - Load Data

In [1]:
import os
import urllib.request

data_url = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
#data_url = 'https://raw.githubusercontent.com/elephantscale/datasets/master/house-prices/house-sales-full.csv'

data_location = os.path.basename(data_url)

if not os.path.exists (data_location):
    print("Downloading : ", data_url)
    urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)


data_location: house-sales-full.csv


In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

data = pd.read_csv(data_location)
data.sample(5)

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
10391,10392,1/24/07,299950,3275810070,Single Family,1/1/07,425600,0.98,"306,716.00",1,...,2.25,3,7,1987,0,0,46000,160000,98023,False
18781,18782,6/22/07,615000,6905200080,Single Family,6/1/07,433200,1.00,"617,839.00",1,...,2.00,4,8,1931,2015,0,416000,258000,98119,False
3803,3804,4/30/07,1495000,1370802235,Single Family,4/1/07,430900,0.99,"1,509,919.00",1,...,3.25,4,10,1978,2006,0,594000,966000,98199,False
17322,17323,6/14/10,241359,6144000030,Single Family,6/1/10,345800,0.79,"303,758.00",1,...,2.50,4,7,1983,0,0,72000,185000,98001,False
4404,4405,5/19/06,292950,1545801510,Single Family,5/1/06,400600,0.92,"318,252.00",1,...,1.75,3,7,1985,0,0,90000,151000,98038,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [3]:
## TODO use 'describe()' function to understand data
data.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [4]:
## any correlated columns?
## TODO : use 'corr' function to verify
data.corr()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data

In [5]:
## TODO : select columns
feature_columns= ['NbrLivingUnits', 'SqFtLot', 'SqFtTotLiving', 'SqFtFinBasement', 'Bathrooms', 'Bedrooms', 
                  'BldgGrade', 'YrBuilt', 'YrRenovated', 'TrafficNoise',  'LandVal', 'ImpsVal', 'NewConstruction']

label_column = ['SalePrice']


In [6]:
X = data[feature_columns]
y = data[label_column]

print ("X.shape = ", X.shape)
print ("y.shape = ", y.shape)

X.shape =  (27063, 13)
y.shape =  (27063, 1)


In [7]:
from sklearn.model_selection import train_test_split

## TODO : split data into train / test, with 20% for test
## hint : 20% is 0.2

X_train,X_test,y_train, y_test = train_test_split(X,y,  test_size=0.2, random_state=123)


print ("x_train :" , X_train.shape )
print ("x_test :", X_test.shape)
print ("y_train :", y_train.shape)
print ("y_test :", y_test.shape)

x_train : (21650, 13)
x_test : (5413, 13)
y_train : (21650, 1)
y_test : (5413, 1)


## Step 4 : Run Regression

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import  LassoLarsIC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

algorithms = [
    LinearRegression(),
    LassoLarsIC(criterion='aic', max_iter=500),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
]

In [9]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

cm_labels = np.unique(y)
    
for algo in algorithms:
    print ()
    print ("============== Running {} ======".format(algo))
    t1 = time.perf_counter()
    model = algo.fit (X_train, np.ravel(y_train))
    t2 = time.perf_counter()
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    t3 = time.perf_counter()
    
    print ("Training time : {:,.2f} secs ({:,.1f} ms)".format ( (t2-t1), (t2-t1)*1e3))
    print ("Prediction time : {:,.2f} secs ({:,.1f} ms)".format ( (t3-t2), (t3-t2)*1e3))
    
    print ("Training R2 : {:,.2f}".format (r2_score(y_train, y_pred_train)))
    print ("Testing R2  : {:,.2f}".format (r2_score(y_test, y_pred_test)))
    print ("Training RMSE : {:,.2f}".format (sqrt(mean_squared_error(y_train, y_pred_train))))
    print ("Testing RMSE  : {:,.2f}".format (sqrt(mean_squared_error(y_test, y_pred_test))))
    
    



============== Running LinearRegression() ======
Training time : 0.03 secs (28.0 ms)
Prediction time : 0.04 secs (36.4 ms)
Training R2 : 0.83
Testing R2  : 0.82
Training RMSE : 140,710.35
Testing RMSE  : 153,564.46

============== Running LassoLarsIC() ======
Training time : 0.04 secs (40.6 ms)
Prediction time : 0.04 secs (38.7 ms)
Training R2 : 0.83
Testing R2  : 0.82
Training RMSE : 140,713.48
Testing RMSE  : 153,537.94

============== Running DecisionTreeRegressor() ======
Training time : 0.16 secs (163.4 ms)
Prediction time : 0.02 secs (16.9 ms)
Training R2 : 0.99
Testing R2  : 0.75
Training RMSE : 24,198.97
Testing RMSE  : 181,788.26

============== Running RandomForestRegressor() ======
Training time : 7.28 secs (7,275.8 ms)
Prediction time : 0.43 secs (433.7 ms)
Training R2 : 0.98
Testing R2  : 0.84
Training RMSE : 50,676.71
Testing RMSE  : 148,615.79

============== Running GradientBoostingRegressor() ======
Training time : 1.82 secs (1,820.2 ms)
Prediction time : 0.03 secs (3